In [1]:
# importing the necessary libraries 
# using selenium lib instead of beautifulsoup to scrape realtime data and to interact with the webpage 
# webdriver acts like a driver for web browser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import seaborn as sns
import time

In [2]:
# set up Chrome options and disabling cookies popup 
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")

# initialize the Selenium WebDriver as the chrome web browser is being controlled by selenium
driver = webdriver.Chrome(options=chrome_options)

def locate_element(driver, locator):
    attempts = 3
    while attempts > 0:
        try:
            return driver.find_element(*locator)
        except Exception as e:
            print(f"Relocating element due to: {e}")
            time.sleep(5)
            attempts -= 1
    raise Exception("Failed to locate element after multiple attempts")

try:
    # open the webpage
    url = 'https://data.nordpoolgroup.com/auction/day-ahead/prices?'\
          'deliveryDate=latest&currency=SEK&aggregation=YearlyAggregate&deliveryAreas=SE3'
    driver.get(url)

    # wait for the table to load
    WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'dx-scrollable-container'))
    )

    # locate the scrollable container as the data is loaded dynamically using js
    scrollable_container = locate_element(driver, (By.CLASS_NAME, 'dx-scrollable-container'))

    # extract and print column names
    header_row = locate_element(driver, (By.CLASS_NAME, 'dx-datagrid-headers'))
    header_cells = header_row.find_elements(By.TAG_NAME, 'td')
    column_names = [cell.text.strip() for cell in header_cells if cell.text.strip()]
    print(f"Column Names: {column_names}")

    # scroll and fetch rows based on faced challenges
    last_row_count = 0
    max_scrolls = 10  
    scrolls = 0

    while scrolls < max_scrolls:
        # fetch rows and ensure all are loaded
        rows = scrollable_container.find_elements(By.CLASS_NAME, 'dx-data-row')
        if len(rows) > last_row_count:
            last_row_count = len(rows)
            scrolls = 0  
        else:
            scrolls += 1  

        # scroll down to load more rows
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_container)
        time.sleep(5)

    # process all loaded rows
    data = []
    for i in range(last_row_count):
        try:
            # locate and interact with the row
            row = locate_element(driver, (By.XPATH, f"(//tr[contains(@class,'dx-data-row')])[{i + 1}]"))
            time.sleep(5)

            # scroll to the row to ensure it's visible
            driver.execute_script("arguments[0].scrollIntoView(true);", row)
            time.sleep(5)  # Add a short delay to ensure the row is fully loaded

            # get all cells (columns) in the row
            cells = row.find_elements(By.XPATH, './/td')

            # check if there are enough cells to extract
            if len(cells) >= 4:
                year = cells[0].text.strip() if cells[0].text else "N/A"
                startdate = cells[1].text.strip() if cells[0].text else "N/A"
                enddate = cells[2].text.strip() if cells[1].text else "N/A"
                value = cells[3].text.strip() if cells[2].text else "N/A"

                # handle hidden cells or cells with colspan as the start date is fixed and hidden
                if 'dx-hidden-cell' in cells[0].get_attribute('class'):
                    year = cells[0].get_attribute('innerHTML').strip()
            
                if 'dx-pointer-events-none' in cells[1].get_attribute('class'):
                    startdate = cells[1].get_attribute('innerHTML').strip()

                data.append([year,startdate, enddate, value])
            else:
                print(f"Row {i}: Skipped due to insufficient cells")

        except Exception as e:
            print(f"Error on row {i}: {e}")

finally:
    # close the WebDriver
    driver.quit()

Column Names: ['Year', 'Delivery Date Start CET', 'Delivery Date End CET', 'SE3 (SEK)']


In [3]:
yearly_df = pd.DataFrame(data, columns=["Year","Start Date", "End Date", "SE3 (SEK)"])
yearly_df

,Year,Start Date,End Date,SE3 (SEK)
0,2024,N/A,2024-12-11,"407,70"
1,2023,N/A,2023-12-31,"589,76"
2,2022,N/A,2022-12-31,"1 378,65"
3,2021,N/A,2021-12-31,"671,60"
4,2020,N/A,2020-12-31,"221,03"
5,2019,N/A,2019-12-31,"405,49"


In [5]:
yearly_df.dtypes

Year          object
Start Date    object
End Date      object
SE3 (SEK)     object
dtype: object

In [10]:
# Replace 'N/A' in 'Start Date' with the first day of the month of 'End Date'


AttributeError: Can only use .dt accessor with datetimelike values

In [13]:
# convert the Delivery start and end Dates to datetime and SE3 (SEK) to float dtypes
yearly_df['Year'] = yearly_df['Year'].astype(int)
yearly_df['Start Date'] = pd.to_datetime(yearly_df['Start Date'],errors = 'coerce')
yearly_df['End Date'] = pd.to_datetime(yearly_df['End Date'],errors = 'coerce')

# Fill 'Start Date' with the first day of the month from 'End Date' where 'Start Date' is 'N/A'
yearly_df['Start Date'] = yearly_df.apply(
    lambda row: row['End Date'].replace(day=1) if pd.isna(row['Start Date']) else row['Start Date'],
    axis=1
)
yearly_df['SE3 (SEK)'] = yearly_df['SE3 (SEK)'].replace({',': '', ' ': ''}, regex=True)
yearly_df['SE3 (SEK)'] = (pd.to_numeric(yearly_df['SE3 (SEK)'], errors='coerce'))
yearly_df

,Year,Start Date,End Date,SE3 (SEK)
0,2024,2024-12-01,2024-12-11,40770
1,2023,2023-12-01,2023-12-31,58976
2,2022,2022-12-01,2022-12-31,137865
3,2021,2021-12-01,2021-12-31,67160
4,2020,2020-12-01,2020-12-31,22103
5,2019,2019-12-01,2019-12-31,40549


In [14]:
yearly_df.to_csv('yearly_df.csv',index = False)